In [1]:
# 주식 데이터 관련 모듈
import pandas_datareader as pdr
import time

# 기본적인 데이터 핸들링
import pandas as pd
import numpy as np

In [2]:
# 데이터프레임 정렬을 위한 리스트 생성
align = ['Stock_Item', 'Title', 'Link', 'Content', 'Date']

# 데이터 로드
df_semiconductor = pd.read_csv('Title_Link_Date/df_semiconductor_content.csv')[align]
df_electric_car_battery = pd.read_csv('Title_Link_Date/df_electric_car_battery_content.csv')[align]
df_petrochemistry = pd.read_csv('Title_Link_Date/df_petrochemistry_content.csv')[align]
df_shipbuilding_shiping = pd.read_csv('Title_Link_Date/df_shipbuilding_shiping_content.csv')[align]
df_steel = pd.read_csv('Title_Link_Date/df_steel_content.csv')[align]

In [3]:
# 기업과 종목코드만 들어있는 딕셔너리 생성
code_dict = {'삼성전자' : '005930', 'SK하이닉스' : '000660', '현대차' : '005380', 'LG에너지솔루션' : '373220',
             'LG화학' : '051910', 'SK이노베이션' : '096770', '롯데케미칼' : '011170', 'S-OIL' : '010950',
             'HD현대중공업' : '329180', '대우조선해양' : '042660', '현대미포조선' : '010620', '삼성중공업' : '010140',
             'HD한국조선해양' : '009540', 'HMM' : '011200', '팬오션' : '028670', 
             '현대제철' : '004020', 'POSCO홀딩스' : '005490', '고려아연' : '010130'}
# 주가 정보를 받아올 시작일과 종료일 설정
start_date = '2022-12-25' # 공휴일등을 생각해서 넉넉하게 수집
end_date = '2023-06-05' 

In [4]:
# 종목별 주가 정보가 담긴 딕셔너리 생성
stock_pirce = {}
for stock, code in code_dict.items():
    # 종목의 일별 종가 가져오기
    df = pdr.DataReader(code, 'naver', start=start_date, end=end_date)[['Close']].reset_index()
    # 전일 종가열 생성
    price_df = pd.concat([df.iloc[1:,:].reset_index(drop=True,),pd.DataFrame({'Pr_Close':df.iloc[:-1,1]})],axis=1)
    # 현종가 -전일 종가 열 생성
    price_df['Diff'] = price_df['Close'].astype('int') - price_df['Pr_Close'].astype('int')
    stock_pirce[stock] = price_df

In [5]:
weekend_to_friday = {}
holiday_list = ['2022-12-30','2023-01-23','2023-01-24','2023-03-01','2023-05-01','2023-05-05','2023-05-29'] #휴장일 리스트 선언

# 휴장일이거나 주말인 경우에는 가장 최근의 개장일로 변환해서 새로운 열 생성
for date in pd.date_range(start=start_date, end=end_date):
    transe_date = date
    # 날짜가 휴장일이 아닐 때까지 전일로 변환
    while transe_date.strftime('%Y-%m-%d') in holiday_list:
        transe_date = transe_date - pd.Timedelta(days=1)
    # 날짜가 주말이면 전 주 금요일로 변환
    if transe_date.dayofweek >= 5:
        last_friday = transe_date - pd.Timedelta(days=transe_date.dayofweek - 4)
        #변환 후에 금요일이라면 전일로 추가 변환
        while last_friday.strftime('%Y-%m-%d') in holiday_list:
            last_friday = last_friday - pd.Timedelta(days=1)
    # 최종적으로 변환된 날짜를 딕셔너리에 추가
    weekend_to_friday[date.strftime('%Y-%m-%d')] = last_friday.strftime('%Y-%m-%d')

In [1]:
# 데이터 프레임에 주가정보 추가 함수
def add_stock_price(df_info, stock_price):
    # 변경된 휴장일 값을 원본 데이터에 추가
    df_info['Adjusted_Date'] = df_info['Date'].apply(lambda x: weekend_to_friday.get(x) if x in weekend_to_friday else x)
    df_info['Adjusted_Date'] = pd.to_datetime(df_info['Adjusted_Date'])
    df_list=[]
    # 종목별로 휴장일 변환을 반영한 날을 기준으로 데이터 merge
    for stock in stock_price.keys():
        df_stock = df_info.loc[df_info.Stock_Item == stock]
        df_add = pd.merge(df_stock, stock_pirce[stock], left_on='Adjusted_Date', right_on='Date', how='left', suffixes=('', '_drop'))
        df_add = df_add.drop(columns=['Date_drop'])  # drop the 'Date' column from the right dataframe
        df_list.append(df_add)
        
    final_df = pd.concat(df_list)
    return final_df

In [7]:
# add_stock_price 함수를 사용해 주가 정보 추가한 뒤 결측값 제거
semiconductor = add_stock_price(df_semiconductor, stock_pirce).dropna()
electric_car_battery = add_stock_price(df_electric_car_battery, stock_pirce).dropna()
petrochemistry = add_stock_price(df_petrochemistry, stock_pirce).dropna()
shipbuilding_shiping = add_stock_price(df_shipbuilding_shiping, stock_pirce).dropna()
steel = add_stock_price(df_steel, stock_pirce).dropna()

In [8]:
# 수집된 데이터 저장
semiconductor.to_csv('Full_data/semiconductor_stock.csv', index=False)
electric_car_battery.to_csv('Full_data/electric_car_battery_stock.csv', index=False)
petrochemistry.to_csv('Full_data/petrochemistry_stock.csv', index=False)
shipbuilding_shiping.to_csv('Full_data/shipbuilding_shiping_stock.csv', index=False)
steel.to_csv('Full_data/steel_stock.csv', index=False)